# Classification Model - FintelHub Capstone

**Objective**: Build and evaluate a classification model to predict binary outcomes in financial data.

**Choose Your Problem:**
- **Option A**: Fraud Detection (PaySim dataset)
- **Option B**: Customer Churn Prediction (Customer Churn dataset)
- **Option C**: Credit Default Prediction (Credit Risk dataset)

---

## Table of Contents
1. [Setup & Imports](#setup)
2. [Data Loading & Exploration](#data-loading)
3. [Data Engineering](#data-engineering)
4. [Model Training](#model-training)
5. [Model Evaluation](#model-evaluation)
6. [Model Saving](#model-saving)
7. [Conclusions](#conclusions)

---
## 1. Setup & Imports <a id='setup'></a>

### Import Required Libraries

In [ ]:
# Core libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# ML libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score, roc_curve
)
from imblearn.over_sampling import SMOTE

# Helper modules (in src folder)
import sys
sys.path.append('../src')
from data_engineering import *
from model_utils import *

# Set random seed for reproducibility
np.random.seed(42)

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8-darkgrid')

print("All libraries imported successfully!")

---
## 2. Data Loading & Exploration <a id='data-loading'></a>

### 2.1 Load Dataset

**Instructions**: Update the file path based on your chosen dataset:
- Fraud Detection: `'../data/raw/fraud_data.csv'`
- Churn Prediction: `'../data/raw/customer_churn.csv'`
- Credit Default: `'../data/raw/credit_risk.csv'`

In [ ]:
# TODO: Update with your chosen dataset path
DATA_PATH = '../data/raw/customer_churn.csv'  # Change this!

# Load data (limit rows for large datasets like fraud data)
# For fraud data, use nrows=100000 to load first 100k rows
df = load_data(DATA_PATH, nrows=None)

# Display first few rows
df.head()

### 2.2 Basic Data Exploration (GUIDED)

In [ ]:
# Dataset shape
print(f"Dataset Shape: {df.shape[0]} rows, {df.shape[1]} columns\n")

# Column information
print("Column Information:")
print(df.info())

# Statistical summary
print("\nStatistical Summary:")
df.describe()

### 2.3 Identify Target Variable

**Target Variable Names:**
- Fraud Detection: `isFraud`
- Churn Prediction: `churn`
- Credit Default: `loan_status`

In [ ]:
# TODO: Set your target column name
TARGET_COLUMN = 'churn'  # Change this based on your dataset!

# Check target distribution
print(f"Target Variable: {TARGET_COLUMN}")
print(f"\nValue Counts:")
print(df[TARGET_COLUMN].value_counts())
print(f"\nPercentage Distribution:")
print(df[TARGET_COLUMN].value_counts(normalize=True) * 100)

# Visualize target distribution
plt.figure(figsize=(8, 5))
df[TARGET_COLUMN].value_counts().plot(kind='bar', color=['#2ecc71', '#e74c3c'])
plt.title(f'Distribution of {TARGET_COLUMN}')
plt.xlabel(TARGET_COLUMN)
plt.ylabel('Count')
plt.xticks(rotation=0)
plt.show()

# Check for class imbalance
imbalance_ratio = df[TARGET_COLUMN].value_counts().min() / df[TARGET_COLUMN].value_counts().max()
print(f"\nClass Imbalance Ratio: {imbalance_ratio:.3f}")
if imbalance_ratio < 0.5:
    print("⚠️ Dataset is imbalanced. We'll need to handle this!")

### 2.4 Check for Missing Values (GUIDED)

In [ ]:
# Check missing values
missing_summary = check_missing_values(df)

### 2.5 Identify Feature Types

In [ ]:
# TODO: Use the helper function to identify numerical and categorical features
feature_types = get_feature_types(df)

numerical_features = feature_types['numerical']
categorical_features = feature_types['categorical']

### 📊 Checkpoint 1
Before proceeding, ensure:
- ✅ Data is loaded successfully
- ✅ You understand the target variable distribution
- ✅ You've identified missing values (if any)
- ✅ You know which features are numerical vs categorical

---
## 3. Data Engineering <a id='data-engineering'></a>

### 3.1 Handle Missing Values

In [ ]:
# TODO: Handle missing values if any were found
# Choose strategy: 'mean', 'median', 'mode', or 'drop'

if len(missing_summary) > 0:
    # Example: Fill numerical columns with median
    df = handle_missing_values(df, strategy='median')
else:
    print("No missing values to handle!")

### 3.2 Drop Irrelevant Columns

**Common columns to drop:**
- ID columns (customer_id, transaction_id, etc.)
- Timestamp columns (unless doing time-series analysis)
- Columns with too many unique values (like names)

In [ ]:
# TODO: Drop irrelevant columns
# Example columns that might need dropping:
# - 'customer_id', 'name', 'nameOrig', 'nameDest', 'step', etc.

columns_to_drop = []  # Add column names here

if columns_to_drop:
    df = df.drop(columns=columns_to_drop)
    print(f"Dropped columns: {columns_to_drop}")

print(f"\nRemaining columns: {df.shape[1]}")
print(df.columns.tolist())

### 3.3 Encode Categorical Variables

In [ ]:
# TODO: Encode categorical variables
# Update categorical_features list if you dropped any columns

# Remove target column from categorical features if present
if TARGET_COLUMN in categorical_features:
    categorical_features.remove(TARGET_COLUMN)

if categorical_features:
    print(f"Encoding categorical features: {categorical_features}")
    # Use 'onehot' for nominal categories (no inherent order)
    # Use 'label' for ordinal categories (has order)
    df = encode_categorical(df, categorical_features, method='onehot')
else:
    print("No categorical features to encode")

print(f"\nShape after encoding: {df.shape}")

### 3.4 Separate Features and Target

In [ ]:
# TODO: Separate features (X) and target (y)
X = df.drop(columns=[TARGET_COLUMN])
y = df[TARGET_COLUMN]

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"\nFeature columns ({len(X.columns)}):")
print(X.columns.tolist())

### 3.5 Train-Test Split (GUIDED)

In [ ]:
# Split data: 80% training, 20% testing
# Use stratify=y to maintain class distribution in imbalanced datasets
X_train, X_test, y_train, y_test = create_train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

print(f"Training set: {X_train.shape[0]} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Test set: {X_test.shape[0]} samples ({X_test.shape[0]/len(X)*100:.1f}%)")

# Verify class distribution is maintained
print(f"\nTraining set class distribution:")
print(y_train.value_counts(normalize=True))
print(f"\nTest set class distribution:")
print(y_test.value_counts(normalize=True))

### 3.6 Feature Scaling

In [ ]:
# TODO: Scale numerical features
# Choose 'standard' (mean=0, std=1) or 'minmax' (range 0-1)

X_train_scaled, X_test_scaled, scaler = scale_features(
    X_train, X_test, 
    method='standard'
)

# Convert back to DataFrame to preserve column names
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

print("\nFeatures scaled successfully!")

### 3.7 Handle Class Imbalance (Optional)

Only needed if your dataset is imbalanced (ratio < 0.5)

In [ ]:
# TODO: Apply SMOTE if dataset is imbalanced
# Uncomment the code below if needed

# smote = SMOTE(random_state=42)
# X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# print(f"Original training set: {X_train_scaled.shape[0]} samples")
# print(f"Resampled training set: {X_train_resampled.shape[0]} samples")
# print(f"\nClass distribution after SMOTE:")
# print(pd.Series(y_train_resampled).value_counts())

# # Use resampled data for training
# X_train_final = X_train_resampled
# y_train_final = y_train_resampled

# If not using SMOTE, use original scaled data
X_train_final = X_train_scaled
y_train_final = y_train

### 📊 Checkpoint 2
Before proceeding to modeling, verify:
- ✅ Missing values are handled
- ✅ Categorical variables are encoded
- ✅ Data is split into train/test sets
- ✅ Features are scaled
- ✅ Class imbalance is addressed (if needed)

---
## 4. Model Training <a id='model-training'></a>

### 4.1 Train Multiple Models (GUIDED)

In [ ]:
# Train three classification models
trained_models = train_classification_models(X_train_final, y_train_final)

print("\nAll models trained successfully!")
print(f"Models: {list(trained_models.keys())}")

### 4.2 Compare Models on Test Set

In [ ]:
# TODO: Compare all models and create comparison dataframe
comparison_df = compare_classification_models(trained_models, X_test_scaled, y_test)

# Display comparison
comparison_df

### 4.3 Select Best Model

In [ ]:
# TODO: Select best model based on F1 score (or other metric)
# You can change the metric if needed (accuracy, precision, recall, roc_auc)

best_model_name = comparison_df.loc[comparison_df['f1'].idxmax(), 'Model']
best_model = trained_models[best_model_name]

print(f"✅ Best Model: {best_model_name}")
print(f"\nBest Model Metrics:")
print(comparison_df[comparison_df['Model'] == best_model_name].to_string(index=False))

---
## 5. Model Evaluation <a id='model-evaluation'></a>

### 5.1 Detailed Evaluation of Best Model

In [ ]:
# Get predictions
y_pred = best_model.predict(X_test_scaled)
y_pred_proba = best_model.predict_proba(X_test_scaled)[:, 1]

# Print detailed classification report
print(f"Classification Report - {best_model_name}")
print("="*60)
print(classification_report(y_test, y_pred))

### 5.2 Confusion Matrix

In [ ]:
# TODO: Plot confusion matrix
plot_confusion_matrix(best_model, X_test_scaled, y_test, best_model_name)

### 5.3 ROC Curve

In [ ]:
# TODO: Plot ROC curve
plot_roc_curve(best_model, X_test_scaled, y_test, best_model_name)

### 5.4 Feature Importance

In [ ]:
# TODO: Get and plot feature importance (for tree-based models)
if hasattr(best_model, 'feature_importances_'):
    feature_importance_df = get_feature_importance(
        best_model, 
        X_train_final.columns, 
        top_n=10
    )
else:
    print(f"{best_model_name} does not have feature importance attribute")

---
## 6. Model Saving <a id='model-saving'></a>

### 6.1 Save the Best Model

In [ ]:
# TODO: Save your best model
model_filename = f"../models/classification_{best_model_name.lower().replace(' ', '_')}.pkl"
save_model(best_model, model_filename)

### 6.2 Save Preprocessing Artifacts

In [ ]:
# TODO: Save scaler for use in production
import joblib
joblib.dump(scaler, '../models/scaler.pkl')
print("✅ Scaler saved to ../models/scaler.pkl")

### 6.3 Save Model Performance Summary

In [ ]:
# Save comparison results
comparison_df.to_csv('../models/classification_model_comparison.csv', index=False)
print("✅ Model comparison saved to ../models/classification_model_comparison.csv")

---
## 7. Conclusions <a id='conclusions'></a>

### 7.1 Summary of Results

**TODO: Write a brief summary of your findings**

1. **Dataset**: [Describe which dataset you used]

2. **Data Challenges**: [Note any issues like missing values, imbalance, etc.]

3. **Best Model**: [State which model performed best and why]

4. **Key Metrics**: [Highlight the most important metrics]

5. **Important Features**: [List top 3-5 features that influenced predictions]

6. **Model Limitations**: [Note any limitations or concerns]

7. **Next Steps**: [Suggest improvements or further analysis]

### 7.2 Test Model Loading

In [ ]:
# Verify model can be loaded
loaded_model = load_model(model_filename)

# Test prediction
test_prediction = loaded_model.predict(X_test_scaled[:5])
print(f"\nTest predictions: {test_prediction}")
print(f"Actual values: {y_test[:5].values}")
print("\n✅ Model loaded and tested successfully!")

---
## 🎉 Congratulations!

You've successfully completed the Classification Model notebook!

**Next Steps:**
1. Review your model performance
2. Try improving the model (feature engineering, hyperparameter tuning)
3. Move on to `02_regression_model.ipynb`
4. Prepare for model deployment to H2O